In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, get_scheduler
import os
import torch
from torch.optim import AdamW
import time
from torch import nn, Tensor
import argparse

In [2]:
class BeaconEmbedding(nn.Module):
    def __init__(self, embedding: nn.Embedding, vocab_size: int, n_embed: int, window_length: int, *args, **kwargs):
        super().__init__()
        self.n_embed = n_embed
        self.b_embed = nn.Parameter(torch.empty(n_embed), requires_grad=True)
        self.window_length = window_length
        self.embedding = embedding
        self.reset_parameters()
    
    def reset_parameters(self):
        nn.init.zeros_(self.b_embed)

    def forward(self, input: Tensor) -> Tensor:
        B, N = input.shape
        regular_embedding = self.embedding(input)
        beacon_tensor = torch.zeros((B, N, self.n_embed))
        beacon_tensor[:, ::self.window_length, :] = self.b_embed
        beacon_tensor = beacon_tensor.to(regular_embedding.device)
        return regular_embedding + beacon_tensor

In [3]:
def generate_beacon_attention_mask_2d(size, window_length=4, direct_window_multiple=1, device=None):
    mask_tensor = torch.zeros((size, size), device=device)
    mask_tensor[:, ::window_length] = 1
    mask_tensor[:, :window_length] = 1 # takes care of attention sinks
    for i in range(size):
        start_index = max(0, i - window_length*direct_window_multiple)
        mask_tensor[i, start_index:i] = 1
        mask_tensor[i, i] = 0
    return mask_tensor.tril()

In [4]:
use_embedding = True
use_custom_attn_mask = True
window_size = 4

In [5]:
num_epochs = 1
batch_size = 128
block_size = 128

# %%
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

In [6]:
from torch.nn import CrossEntropyLoss
import torch

def causal_lm_loss(inputs, logits, alpha=1.0):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss(reduce=False)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
    # Calculate average
    loss = loss_per_sample.mean()
    return loss

In [7]:
from datasets import load_dataset

def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=block_size,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == block_size:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

ds = load_dataset("roneneldan/TinyStories")
tokenized_ds = ds.map(tokenize, batched=True, batch_size=batch_size, num_proc=4, remove_columns=ds["train"].column_names)
tokenized_ds.set_format("torch")

Repo card metadata block was not found. Setting CardData to empty.


In [8]:
from torch.utils.data.dataloader import DataLoader

train_dataloader = DataLoader(tokenized_ds["train"], batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(tokenized_ds["validation"], batch_size=batch_size)

print(len(eval_dataloader))

212


In [9]:
weight_decay = 0.1

def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [10]:
model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-1m')
model = model.to(device)
if use_embedding:
    beacon_embedding = BeaconEmbedding(embedding=model.get_input_embeddings(), vocab_size=model.config.vocab_size, n_embed=model.config.hidden_size, window_length=window_size)
    beacon_embedding = beacon_embedding.to(model.device)
    model.set_input_embeddings(beacon_embedding)
model.config

GPTNeoConfig {
  "_name_or_path": "roneneldan/TinyStories-1m",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      4
    ]
  ],
  "bos_token_id": 50256,
  "classifier_dropout": 0.1,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 64,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 16,
  "num_layers": 8,
  "resid_dropout": 0,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.43.3",
  

In [11]:
beacon_attention_mask = generate_beacon_attention_mask_2d(block_size, window_length=window_size, device=device)
print("Attention mask: ", beacon_attention_mask[:18, :18])

Attention mask:  tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],

In [12]:
optimizer = AdamW(get_grouped_params(model), lr=5e-5)
num_training_steps = num_epochs * len(train_dataloader)

def evaluate():
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            if use_custom_attn_mask:
                # print("Shape: ", batch["input_ids"].shape, beacon_attention_mask.shape)
                X, Y = batch["input_ids"].shape
                outputs = model(batch["input_ids"], labels=batch["input_ids"], attention_mask=beacon_attention_mask[:X, :Y])
            else:
                outputs = model(batch["input_ids"], labels=batch["input_ids"])

        losses.append(accelerator.gather(outputs.loss))
    # print(losses)
    loss = torch.mean(torch.Tensor(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()

In [13]:
from accelerate import Accelerator
import wandb

accelerator = Accelerator() # Logging with wandb here isn't working as expected for some reason
wandb.init(
    project="beacon-attention-1m",
    config={
        "use_custom_embedding": use_embedding,
        "use_custom_attn_mask": use_custom_attn_mask,
        "window_size": window_size,
    },
    # mode="disabled"
)

wandb: Currently logged in as: vdaita. Use `wandb login --relogin` to force relogin


In [14]:
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

model_name = f"{'beacon_embed' if use_embedding else 'no_beacon_embed'}_{'beacon_attn_mask' if use_custom_attn_mask else 'regular_attn_mask'}_window_size_{window_size}_model"
output_dir = f"./models/{model_name}"
if not(os.path.exists(output_dir)):
    os.makedirs(output_dir)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [15]:
from tqdm import tqdm

gradient_accumulation_steps = 8
eval_steps = 5000

model.train()

completed_steps = 0
step_start_time = time.perf_counter()

eval_loss, perplexity = evaluate()

for epoch in range(num_epochs):
    for step, batch in tqdm(enumerate(train_dataloader, start=1), total=num_training_steps):
        X, Y = batch['input_ids'].shape
        if use_custom_attn_mask:
            # print("Shape: ", batch["input_ids"].shape, beacon_attention_mask.shape)
            logits = model(input_ids=batch["input_ids"], attention_mask=beacon_attention_mask[:X, :Y]).logits
        else:
            logits = model(input_ids=batch["input_ids"]).logits
        loss = causal_lm_loss(batch["input_ids"], logits)

        if step % 100 == 0:
            step_end_time = time.perf_counter()
            train_update = {
                "samples": step * batch_size,
                "steps": completed_steps,
                "loss/train": loss.item(), # * gradient_accumulation_steps,
                "step_time": step_end_time - step_start_time
            }
            accelerator.print(train_update)
            wandb.log(train_update)
            step_start_time = step_end_time

        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)

        if step % gradient_accumulation_steps == 0:
            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
        if (step % 1000) == 0:
            eval_loss, perplexity = evaluate()
            eval_update = {"loss/eval": eval_loss, "perplexity": perplexity}
            accelerator.print(eval_update)
            wandb.log(eval_update)
            model.train()
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
        
        end_time = time.perf_counter()

  0%|                                                                          | 0/21210 [00:00<?, ?it/s]/home/vijay/.local/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
  0%|▎                                                               | 100/21210 [00:15<53:46,  6.54it/s]

{'samples': 12800, 'steps': 12, 'loss/train': 3.706055164337158, 'step_time': 27.49883888103068}


  1%|▌                                                               | 200/21210 [00:30<53:56,  6.49it/s]

{'samples': 25600, 'steps': 24, 'loss/train': 3.6019177436828613, 'step_time': 15.298237908631563}


  1%|▉                                                               | 300/21210 [00:45<52:49,  6.60it/s]

...

 99%|█████████████████████████████████████████████████████████████▋| 21100/21210 [57:59<00:16,  6.52it/s]

{'samples': 2700800, 'steps': 2637, 'loss/train': 3.113971710205078, 'step_time': 27.3056007809937}


100%|█████████████████████████████████████████████████████████████▉| 21200/21210 [58:15<00:01,  6.44it/s]

{'samples': 2713600, 'steps': 2649, 'loss/train': 3.1194911003112793, 'step_time': 15.376722183078527}


100%|██████████████████████████████████████████████████████████████| 21210/21210 [58:16<00:00,  6.07it/s]


In [16]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
wandb.finish()

loss/eval,█▆▅▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
loss/train,█▆▅▃▃▃▄▃▃▃▃▃▂▃▃▃▃▂▂▂▂▂▃▂▁▂▁▂▂▂▂▂▁▁▂▂▂▂▂▂
perplexity,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
samples,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
step_time,▅▁▃▄▅▅▇█▄██▂▄▇▄▄▃▄▆▃▅▇█▄▃▆▄▄▃▃▇▄▆▇█▄▅▆▃▇
steps,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/eval,3.10327
loss/train,3.11949
perplexity,22.27072
samples,2713600
step_time,15.37672
